In [13]:
from sys import displayhook

import pandas as pd
from pandas import json_normalize
import json
from pathlib import Path

DATA_ITEM = 'youtube'

parquet_path = Path().resolve() / DATA_ITEM / 'parquet'
raw_path = Path().resolve() / DATA_ITEM / 'raw'

# get the latest parquet file
parquet_files = list(parquet_path.glob('*.parquet'))
latest_parquet_file = max(parquet_files, key=lambda x: x.stat().st_mtime)

# read the latest parquet file
df = pd.read_parquet(latest_parquet_file)

# count the number of rows
print(f"Number of rows: {len(df)}")


Number of rows: 220


In [ ]:
# PARQUET SCHEMA
from fastparquet import ParquetFile
pf = ParquetFile(latest_parquet_file)
print(pf.schema)


In [ ]:

# Convert JSON string columns to dictionaries, handling None values
json_columns = [
    "snippet",
    "contentDetails",
    "status",
    "statistics",
    "player",
    "topicDetails"
]

for col in json_columns:
    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notna(x) else {})
    
dict_df_obj = df.to_dict('records')


In [15]:

# Now normalize the nested structures
json_normalized_df = pd.json_normalize(
    df.to_dict('records'),
    sep='_',
    max_level=None
)

# Preview the flattened columns
print("Flattened columns:", json_normalized_df.columns.tolist())


Flattened columns: ['id', 'kind', 'etag', 'transcript', 'snippet_publishedAt', 'snippet_channelId', 'snippet_title', 'snippet_description', 'snippet_channelTitle', 'snippet_categoryId', 'snippet_liveBroadcastContent', 'contentDetails_duration', 'contentDetails_dimension', 'contentDetails_definition', 'contentDetails_caption', 'contentDetails_licensedContent', 'contentDetails_projection', 'status_uploadStatus', 'status_privacyStatus', 'status_license', 'status_embeddable', 'status_publicStatsViewable', 'status_madeForKids', 'statistics_viewCount', 'statistics_likeCount', 'statistics_favoriteCount', 'statistics_commentCount', 'player_embedHtml', 'topicDetails_topicCategories']
